In [ ]:
# Import dependencies
import os
import wfdb
import numpy as np
from pydub import AudioSegment
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import parselmouth

In [ ]:
# CONSTANTS

# Since audio data, need signed bit integer
# According to dataset: 32-bit
SIGNED_32BIT = 2**31 - 1

# According to dataset: 8000Hz
SAMPLING_FREQ = 8000

# Since 32-bit, 8bits/byte
SAMPLE_WIDTH = 4

## Import data

In [ ]:
# Raw dataset file path
rawdata_path = "resources/voiced_dataset/"

# Get all the files in the directory
files = os.listdir(rawdata_path)

# Create lists to check dataset
voice_info = []
voice_file = []
audio_file = []

# Loop through each file
for file in files:
    
    # Look through voice files only
    if file.startswith("voice"):
        
        # Isolate the info text files
        if file.endswith("-info.txt"):
            voice_id = file.split("-")[0]
            voice_info.append(voice_id)
        
        # Isolate the voice text files
        elif file.endswith(".txt"):
            voice_id = file.split(".")[0]
            voice_file.append(voice_id)
            
        # Isolate the audio files
        elif file.endswith(".hea"):
            voice_id = file.split(".")[0]
            audio_file.append(voice_id)

# Confirm each sample has a signal and info file
print(f'Info file: {len(set(voice_info))}')
print(f'Signal file: {len(set(voice_file))}')
print(f'Audio file: {len(set(audio_file))}')

### Convert wfdb to wav

In [ ]:
# Convert to a wav file
for file in files:
    
    # Extract .hea files only
    if file.startswith("voice") and file.endswith(".hea"):
        voice_id = file.split(".")[0]
        
        # Get the record name
        record_name = rawdata_path + voice_id
        
        # Read the wfdb record
        record = wfdb.rdrecord(record_name)
        
        # Get the signal data and sampling freq (fs)
        signal_data = record.p_signal
        fs = record.fs
        
        # Find the min and max of the signal data
        signal_min = np.min(signal_data)
        signal_max = np.max(signal_data)
        
        # Normalise and scale to 32-bit range
        normalised_signal = (signal_data - signal_min) / (signal_max - signal_min)
        normalised_signal = (SIGNED_32BIT * normalised_signal).astype(np.int32)
        
        # Create an AudioSegment from the normalised signal data
        audio_segment = AudioSegment(
            normalised_signal.tobytes(),
            frame_rate = fs,
            sample_width = SAMPLE_WIDTH,
            channels = 1 # mono, not stereo
        )
        
        # Save as a wav file
        output_filename = f'resources/audio_files/{voice_id}.wav'
        audio_segment.export(output_filename, format='wav')

### Convert info text file to DataFrame

In [ ]:
# Display an example of the info text file
sample_file = rawdata_path + "voice001-info.txt"

# Read the text file
with open(sample_file, 'r') as file:
    for line in file:
        print(line.strip())

In [ ]:
# Initialise the list to hold dictionaries
metadata_list = []

# Parse the data in the info files
for file in files:
    
    # Initialise the dictionary to store the info
    metadata_dict = dict()
    
    # Look through info files only
    if file.startswith("voice") and file.endswith("-info.txt"):
        
        # Read the text file
        with open(rawdata_path + file, 'r') as file:
            for line in file:
                
                # Split each line into a key-value pair using delimiter
                key, value = map(str.strip, line.split("\t"))
                
                # Ignore the empty lines by checking whitespaces
                if not line.strip():
                    continue
                else:
                    # Remove the colon
                    key = key.replace(":", "")
                    
                    # Load the data to a dictionary
                    metadata_dict[key] = value
                    
            # Append the dictionary to the list
            metadata_list.append(metadata_dict)

# Convert the list to a DataFrame
metadata_df = pd.DataFrame(metadata_list)
metadata_df.head()

### Signal Visualisations

In [ ]:
# Retrieve the audio files
audio_path = "resources/audio_files/"
audio_files = os.listdir(audio_path)

In [ ]:
# Display the first 3 waveforms
for file in audio_files[:3]:
    
    # Load the file
    y, sr = librosa.load(
        audio_path + file, # full file path
        sr = None # preserve sampling rate
    )
    
    # Plot the waveform
    librosa.display.waveshow(y, sr=sr)
    
    # Add labels
    plt.title(f'Waveform [{file.split(".")[0]}]')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    
    # Display the waveform
    plt.show()

In [ ]:
# Display the first 3 spectrograms
for file in audio_files[:3]:
    
    # Load the file
    y, sr = librosa.load(
        audio_path + file, # full file path
        sr = None # preserve sampling rate
    )
    
    # Plot the spectrogram
    D = librosa.amplitude_to_db(
        np.abs(librosa.stft(y)),
        ref = np.max
    )

    # Plot the spectrogram
    librosa.display.specshow(
        D,
        sr = sr,
        x_axis='time',
        y_axis='log' # can also choose: linear
    )
    
    # Add labels
    plt.title(f'Waveform [{file.split(".")[0]}]')
    plt.colorbar(format='%+2.0f dB')
    
    plt.show()

## Formants using Parselmouth

In [ ]:
for file in audio_files[:3]:
    
    # Load the audio file
    sound = parselmouth.Sound(audio_path + file)
    
    # Extract the formants - Burg's method
    formants = sound.to_formant_burg()
    print(formants)
    
    # Define fundamental freq limits
    f0_min = 50
    f0_max = SAMPLING_FREQ / 2 # Nyquist frequency
    
    # Compute the occurrences of periodic instances
    pointProcess = parselmouth.praat.call(
        sound,
        "To PointProcess (periodic, cc)",
        f0_min,
        f0_max
    )
    
    # Compute the formants
    formants = parselmouth.praat.call(
        sound,
        "To Formant (burg)",
        0.0025,
        5,
        5000,
        0.025,
        50
    )
    
    numPoints = parselmouth.praat.call(pointProcess, "Get number of points")
    f1_list = []
    f2_list = []
    f3_list = []
    for point in range(0, numPoints):
        point += 1
        t = parselmouth.praat.call(pointProcess, "Get time from index", point)
        f1 = parselmouth.praat.call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = parselmouth.praat.call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = parselmouth.praat.call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        
    print(len(f1_list), len(f2_list), len(f3_list))

## Harmonics using Librosa

In [ ]:
# Get the fundamental frequency (f0)
for file in audio_files[:4]:
    print(file)
    
    # Load the file
    y, sr = librosa.load(
        audio_path + file, # full file path
        sr = None # preserve sampling rate
    )

    # Estimate f0
    f0, voicing, voicing_probability = librosa.pyin(
        y = y,
        sr = sr,
        fmin = 50,
        fmax = sr / 2 # Nyquist frequency
    )
    
    # Magnitude spectrogram - freq content of signal over time
    S = np.abs(librosa.stft(y))
    
    # Get the frequency bins
    freqs = librosa.fft_frequencies(sr=sr)
    print(len(freqs), freqs)
    
    # Harmonic analysis (first 12)
    harmonics = np.arange(1, 13)
    f0_harmonics = librosa.f0_harmonics(
        S,
        freqs = freqs,
        f0 = f0,
        harmonics = harmonics
    )
    print(len(f0_harmonics), len(f0_harmonics[0]))
    
    fig, ax =plt.subplots(nrows=2, sharex=True)
    librosa.display.specshow(
        librosa.amplitude_to_db(S, ref=np.max),
        x_axis='time',
        y_axis='log',
        ax=ax[0]
    )
    times = librosa.times_like(f0)
    for h in harmonics:
        ax[0].plot(times, h * f0, label=f"{h}*f0")
    ax[0].legend(ncols=4, loc='lower right')
    ax[0].label_outer()

    librosa.display.specshow(
        librosa.amplitude_to_db(f0_harmonics, ref=np.max),
        x_axis = 'time',
        ax = ax[1]
    )
    ax[1].set_yticks(harmonics-1)
    ax[1].set_yticklabels(harmonics)
    ax[1].set(ylabel='Harmonics')

In [ ]:
metadata_df.loc[metadata_df['ID'].isin(['voice094', 'voice080', 'voice057', 'voice043'])]

In [ ]:
for file in audio_files[:3]:
    
    # Load the file
    y, sr = librosa.load(
        audio_path + file, # full file path
        sr = None # preserve sampling rate
    )

    # Extract harmonics
    harmonic, percussive = librosa.effects.hpss(y)
    # print(len(harmonic), harmonic)
    
    # Extract pitches
    # pitches, magnitudes = librosa.core.piptrack(
    #     y = harmonic,
    #     sr = sr
    # )
    
    # Get the indices
    hps = librosa.effects.harmonic(y)
    print(len(hps), hps)
    print(len(harmonic), harmonic)
    

    
    # Create the time axis
    # time = np.arange(0, len(y)) / sr
    
    # Plot the original waveform and harmonics
    # plt.plot(time, y, label = 'Original Signal')
    # plt.plot(
    #     time,
    #     harmonic,
    #     label = 'Harmonic Component',
    #     linestyle = '--'
    # )
    # plt.show()